<a href="https://colab.research.google.com/github/david-samtani/bert-ai-fake-news-detection/blob/main/bert_fake_news_detection_kaggle2018_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CHECKPOINT_FOLDER = '/content/drive/MyDrive/FakeNews/Checkpoints'
!ls {CHECKPOINT_FOLDER}

10  100  12  14  16  18  2  200  300  4  6  8


# Kaggle
[Dataset](https://www.kaggle.com/competitions/fake-news/data?select=train.csv)

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle.json" with length 68 bytes


In [ ]:
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c fake-news
! unzip fake-news.zip

 77% 36.0M/46.5M [00:00<00:00, 123MB/s] 
100% 46.5M/46.5M [00:00<00:00, 104MB/s]
Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")
df = df[df.text.notna()]
columns = list(df.columns)
columns[-1] = 'labels'
df.columns = columns
df = df[df.text.apply(lambda text: text!='')]
df

,id,title,author,text,labels
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
test_df = pd.read_csv('test.csv')
test_df = test_df[test_df.text.apply(lambda text: text!='')]
test_df = test_df[test_df.text.notna()]
test_df

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

dataset = Dataset.from_pandas(df)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenization(example):
  try:
    tokens = tokenizer(example["text"], padding = 'max_length', truncation = True)
    return tokens
  except:
    with open('debug.txt', 'w+') as debug:
      debug.write(f'example: {example}\n')

In [ ]:
import torch
dataset = dataset.map(tokenization, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataset.format['type']

  0%|          | 0/21 [00:00<?, ?ba/s]

'torch'

In [ ]:
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask"])
test_dataset.format['type']
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10)

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True)
val_test_dataset = dataset["test"].train_test_split(test_size=0.5)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset['train'], batch_size=10, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_test_dataset['train'], batch_size=10)
test_loader = torch.utils.data.DataLoader(val_test_dataset['test'], batch_size=10)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity").cuda()

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]



Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
import os
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)
for epoch in range(1000):
  print(f'epoch: {epoch + 1}')
  total_train_loss = 0
  best_val_loss = math.inf
  for batch in train_loader:
    batch = {key: value.to(device) for key, value in batch.items()}
    predictions = model(**batch)
    predictions.loss.backward()
    total_train_loss += predictions.loss.item()
    optimizer.step()
  print(f'train loss: {total_train_loss}')
  total_val_loss = 0
  for batch in val_loader:
    batch = {key: value.to(device) for key, value in batch.items()}
    with torch.no_grad():
      predictions = model(**batch)
    total_val_loss += predictions.loss.item()
  print(f'validation loss: {total_val_loss}')
  if total_val_loss < best_val_loss:
    best_val_loss = total_val_loss
    print('new minimum val loss achieved')

  print('\n')

  if (epoch + 1)%100 == 0:
    save_dir = f'{CHECKPOINT_FOLDER}/{epoch + 1}'
    os.makedirs(save_dir, exist_ok=True)
    model.save_pretrained(save_dir)
    print(f'save model to {save_dir}')

epoch: 1
train loss: 1233.3744359612465
validation loss: 167.68423753976822
new minimum val loss achieved


epoch: 2
train loss: 1796.9331555366516
validation loss: 321.61969769001007
new minimum val loss achieved


epoch: 3
train loss: 2035.2445878610015
validation loss: 332.046563655138
new minimum val loss achieved


epoch: 4
train loss: 2061.3676747977734
validation loss: 221.90622237324715
new minimum val loss achieved


epoch: 5
train loss: 2128.8408045470715
validation loss: 375.6802750825882
new minimum val loss achieved


epoch: 6
train loss: 2263.36984132044
validation loss: 339.53024357557297
new minimum val loss achieved


epoch: 7
train loss: 2343.746106058359
validation loss: 402.4860179424286
new minimum val loss achieved


epoch: 8


In [ ]:
epoch_to_test = 4
loaded_model = BertForSequenceClassification.from_pretrained(f"{CHECKPOINT_FOLDER}/{epoch_to_test}").cuda()

In [ ]:
device = "cuda:0"
total_test_loss = 0
total_correct = 0
for batch in test_loader:
  batch = {key: value.to(device) for key, value in batch.items()}
  with torch.no_grad():
    predictions = loaded_model(**batch)
    preds = (predictions.logits[:,0] < predictions.logits[:,1]).long()
    total_correct += (preds == batch["labels"]).sum().item()
    total_test_loss += predictions.loss.item()
print(f'test loss: {total_test_loss}\n')
print(f'accuracy: {total_correct/len(test_loader.dataset)}\n')

test loss: 151.88441535830498

accuracy: 0.5743861338468945



In [ ]:
# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels)

labels = torch.tensor(1)
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]



In [ ]:
# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = BertForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels, problem_type="multi_label_classification"
)

labels = torch.nn.functional.one_hot(torch.tensor([predicted_class_id]), num_classes=num_labels).to(
    torch.float
)
loss = model(**inputs, labels=labels).loss
loss.backward()